In [1]:
import numpy as np
from numpy import linalg as la
from scipy import integrate as sci
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)

## Problem 1

### a)

In [2]:
mu_E = 398600
a = 7000
e = 0.05
i = np.radians(35)
big_omega = np.radians(100)
lil_omega = np.radians(30)
M = np.radians(0)

# a = (6678 + 9940)/2
# e = (9940 - 6678)/(9940+6678)
# i = np.radians(28)
# big_omega = np.radians(45)
# lil_omega = np.radians(30)
# f = np.radians(40)

J_2 = 0.00108263
R_E = 6378

In [3]:
def getE(M):
    E_curr = M
    err = 1
    tol = 1e-10
    while (abs(err) > tol):
        err = M - E_curr + e * np.sin(E_curr)
        derrdE = -1 + e * np.cos(E_curr)
        E_next = E_curr - err / derrdE
        E_curr = E_next
    return E_curr

In [4]:
#get eccentric anomaly
E = getE(M)
E

0.0

In [5]:
#get f from E (unnecessary)
f = 2 * np.arctan2(np.tan(E/2) * np.sqrt(1 + e), np.sqrt(1-e))
f

0.0

In [6]:
#calculate orbital elements
h = np.sqrt(mu_E * a * (1-e**2))
h

52756.274508346396

In [7]:
theta = lil_omega + f
theta

0.5235987755982988

In [8]:
r_mag = h**2 / (mu_E * ( 1 + e * np.cos(f) ) )
r_mag

6650.0

In [9]:
#calculate initial radius and velocity
r_0 = r_mag * np.array([
    np.cos(theta) * np.cos(big_omega) - np.cos(i) * np.sin(big_omega) * np.sin(theta),
    np.cos(theta) * np.sin(big_omega) + np.cos(i) * np.cos(big_omega) * np.sin(theta),
    np.sin(i) * np.sin(theta)
])
r_0

array([-3682.35354532,  5198.61357391,  1907.14165087])

In [10]:
v_0 = -(mu_E/h) * np.array([
    np.cos(big_omega) * (np.sin(theta) + e * np.sin(lil_omega)) + np.sin(big_omega) * (np.cos(theta) + e * np.cos(lil_omega)) * np.cos(i),
    np.sin(big_omega) * (np.sin(theta) + e * np.sin(lil_omega)) - np.cos(big_omega) * (np.cos(theta) + e * np.cos(lil_omega)) * np.cos(i),
    -(np.cos(theta) + e * np.cos(lil_omega)) * np.sin(i)
])
v_0

array([-4.85361623, -4.88365245,  3.94070938])

### b)

In [11]:
#time range of 10 initial periods
t_f = 10 * 2 * np.pi * np.sqrt(a**3 / mu_E)

In [12]:
#linearize system to be the position vector and the velocity vector on top of each other
#change in x is velocity on top of acceleration, which we calculate using uniform and perturbed gravity
def dxdt(t,x):
    xdot = np.zeros_like(x)
    r_mag = la.norm(x[0:3])

    p = 1.5 * (J_2 * mu_E * R_E**2 / r_mag**4) * np.array([
        (x[0] / r_mag) * (5 * (x[2] / r_mag)**2 - 1),
        (x[1] / r_mag) * (5 * (x[2] / r_mag)**2 - 1),
        (x[2] / r_mag) * (5 * (x[2] / r_mag)**2 - 3)
    ])
    
    xdot[0] = x[3]
    xdot[1] = x[4]
    xdot[2] = x[5]
    xdot[3] = -mu_E * x[0] / r_mag**3 + p[0]
    xdot[4] = -mu_E * x[1] / r_mag**3 + p[1]
    xdot[5] = -mu_E * x[2] / r_mag**3 + p[2]

    return xdot

In [13]:
#calculate initial state
x_0 = np.concatenate((r_0,v_0), axis=0)
x_0

array([-3682.35354532,  5198.61357391,  1907.14165087,    -4.85361623,
          -4.88365245,     3.94070938])

In [14]:
#use numerical integrator to solve for state over time
soln = sci.solve_ivp(dxdt, (0,t_f), x_0, atol = 1e-10, rtol=1e-13)
t = soln.t
x = soln.y

In [15]:
print(x.shape[1])

6643


In [16]:
#1.3
#calculate initial radius and perturbation
r_mag = la.norm(x_0[0:3])
1.5 * (J_2 * mu_E * R_E**2 / r_mag**4) * np.array([
        (x_0[0] / r_mag) * (5 * (x_0[2] / r_mag)**2 - 1),
        (x_0[1] / r_mag) * (5 * (x_0[2] / r_mag)**2 - 1),
        (x_0[2] / r_mag) * (5 * (x_0[2] / r_mag)**2 - 3)
    ])

array([ 0.00000439, -0.0000062 , -0.00001   ])

In [99]:
#1.4
#retrieve final position values
print(x[0][-1], x[1][-1] , x[2][-1])

-3017.9873979961426 5423.087106428289 3220.143207271979


In [100]:
#1.5
#retrieve final velocity values
print(x[3][-1], x[4][-1] , x[5][-1])

-7.241215495427802 -3.4973840739493234 1.1724468192364914


In [115]:
#plot orbit

# fig = plt.figure(figsize=(8,8))
# ax = fig.add_subplot(111, projection='3d')
# ax.plot(x[0]/R_E, x[1]/R_E, x[2]/R_E, color='r', label='Orbit')
# # ax.plot(0,0,0, marker="o", markersize=1, color='b', label='Earth (not to scale)')
# # ax.plot(0,0,0, marker="o", markersize=50, color='b')
# ax.set_xticks([-1, -0.5, 0, 0.5, 1])
# ax.set_xlabel("x (RE)")
# ax.set_ylabel("y (RE)")
# ax.set_zlabel("z (RE)")
# ax.axis('equal')
# ax.set_box_aspect(aspect=None,zoom=0.8)
# ax.view_init(elev=20)
# plt.title('J2 Perturbed Orbit Over 10 Periods')
# plt.legend()
# plt.tight_layout()
# plt.show()

In [130]:
fig.savefig('J2 Perturbed Orbit Over 10 Periods.png', facecolor='white', transparent=False)

In [102]:
#1.7-11
a_orbit = np.zeros_like(t)
e_orbit = np.zeros_like(t)
i_orbit = np.zeros_like(t)
lil_omega_orbit = np.zeros_like(t)
big_omega_orbit = np.zeros_like(t)

In [103]:
#calculate orbital elemets at each time
x_T = x.T
K = np.array([0,0,1])
I = np.array([1,0,0])

for j in range(t.size):
    r = x_T[j][0:3]
    v = x_T[j][3:6]
    r_mag = la.norm(r)
    v_mag = la.norm(v)
    h = np.cross(r,v)
    e = ( np.cross(v,h) / mu_E ) - r / r_mag
    n = np.cross(K,h)
    i = np.arccos( np.dot(h,K) / la.norm(h))
    lil_omega = np.arccos( np.dot(n,e) / (la.norm(e) * la.norm(n)) )
    big_omega = np.arccos(np.dot(n,I)/ la.norm(n))
    a = 1 / ( (2/r_mag) - ((v_mag**2)/mu_E) )
    a_orbit[j] = a
    e_orbit[j] = la.norm(e)
    i_orbit[j] = i
    lil_omega_orbit[j] = lil_omega
    big_omega_orbit[j] = big_omega

In [104]:
#retrieve final values of elements
print(a_orbit[-1])
print(e_orbit[-1])
print(np.degrees(i_orbit[-1]))
print(np.degrees(lil_omega_orbit[-1]))
print(np.degrees(big_omega_orbit[-1]))

8308.457194518049
0.19624417055956547
27.99755795854984
35.40934145808634
41.69616687830313


In [114]:
#1.12
#plot and save elements

# fig = plt.figure(figsize=(8,4)) 
# plt.plot(t/3600, a_orbit, color='b')
# plt.xlabel("time (hr)")
# plt.ylabel("a (km)")
# plt.axis('equal')
# plt.title('Semimajor Axis (a) Over 10 Periods')
# plt.show()

# fig.savefig('Semimajor Axis (a) Over 10 Periods.png', facecolor='white', transparent=False)

In [113]:
# fig = plt.figure(figsize=(8,4)) 
# plt.plot(t/3600, e_orbit, color='b')
# plt.xlabel("time (hr)")
# plt.ylabel("e")
# plt.title('Eccentricity (e) Over 10 Periods')
# plt.show()

# fig.savefig('Eccentricity (e) Over 10 Periods.png', facecolor='white', transparent=False)

In [112]:
# fig = plt.figure(figsize=(8,4)) 
# plt.plot(t/3600, np.degrees(i_orbit), color='b')
# plt.xlabel("time (hr)")
# plt.ylabel("i (deg) ")
# plt.title('Inclination (i) Over 10 Periods')
# plt.show()

# fig.savefig('Inclination (i) Over 10 Periods.png', transparent=False)

In [111]:
# fig = plt.figure(figsize=(8,4)) 
# plt.plot(t/3600, np.degrees(lil_omega_orbit), color='b')
# plt.xlabel("time (hr)")
# plt.ylabel("w (deg)")
# plt.title('Argument of Periapsis (w) Over 10 Periods')
# plt.show()

# fig.savefig('Argument of Periapsis (w) Over 10 Periods.png', transparent=False)

In [110]:
# fig = plt.figure(figsize=(8,4)) 
# plt.plot(t/3600, np.degrees(big_omega_orbit), color='b')
# plt.xlabel("time (hr)")
# plt.ylabel("RAAN (deg)")
# plt.title('Right Ascension of Ascending Node (RAAN) Over 10 Periods')
# plt.show()

# fig.savefig('Right Ascension of Ascending Node (RAAN) Over 10 Periods.png', transparent=False)

## Problem 2

In [46]:
#time, angles, and locations of observations

t1 = 0 #s
t2 = 118.1 #s
t3 = 237.58 #s
alpha_1 = np.radians(43.537)
alpha_2 = np.radians(54.420)
alpha_3 = np.radians(64.318)
delta_1 = np.radians(-8.7883)
delta_2 = np.radians(-12.074)
delta_3 = np.radians(-15.105)
R_1 = np.array([
    3489.83, 3430.17, 4078.54
])
R_2 = np.array([
    3460.13, 3460.13, 4078.54
])
R_3 = np.array([
    3429.86, 3490.13, 4078.54
])

#represent R as a matrix to access easier
R = np.array([R_1, R_2, R_3])

In [59]:
#2.1
#calculate time parameters, and direction of satellite from observations using angles

tau_1 = t1 - t2
tau_3 = t3 - t2
tau = t3 - t1
rho_1_hat = np.array([ np.cos(delta_1) * np.cos(alpha_1), np.cos(delta_1) * np.sin(alpha_1), np.sin(delta_1) ])
rho_2_hat = np.array([ np.cos(delta_2) * np.cos(alpha_2), np.cos(delta_2) * np.sin(alpha_2), np.sin(delta_2) ])
rho_3_hat = np.array([ np.cos(delta_3) * np.cos(alpha_3), np.cos(delta_3) * np.sin(alpha_3), np.sin(delta_3) ])

#calculate cross products for later equations
p = np.array([
    np.cross(rho_2_hat, rho_3_hat), np.cross(rho_1_hat, rho_3_hat), np.cross(rho_1_hat, rho_2_hat)
])

# print(rho_1_hat)
# print(rho_2_hat)
# print(rho_3_hat)

In [60]:
#2.2
#calculate parameter D_0
D_0 = np.dot(rho_1_hat,p[0])

In [62]:
#represent D scalars as a matrix. zero-indexing makes this tricky
D = np.zeros((3,3))
for i in range(3):
    for j in range(3):
        D[i][j] = np.dot(R[i],p[j])
# print(D)

In [63]:
#calculate A, B, E, a, b, c parameters
A = (1/D_0) * ( -D[0][1] * (tau_3/ tau) + D[1][1] + D[2][1] * (tau_1/tau) )
B = ( 1 / ( 6 * D_0 ) ) * ( D[0][1] * (tau_3**2 - tau**2) * (tau_3/tau) + D[2][1] * (tau**2 - tau_1**2) * (tau_1/tau) )
E = np.dot(R[1], rho_2_hat)
a = - (A**2 + 2 * A * E + np.dot(R[1], R[1]))
b = - 2 * mu_E * B * (A + E)
c = - mu_E**2 * B**2

In [64]:
#solve the polynomial, choose the reasonable root for r_2_mag
soln = np.roots([1,0,a,0,0,b,0,0,c])
print(soln)

[-6584.94922689+4597.60581268j -6584.94922689-4597.60581268j
 -7027.79904045   +0.j          9228.07608707   +0.j
   825.46197859+6881.21042704j   825.46197859-6881.21042704j
  4659.34872499+4901.85635182j  4659.34872499-4901.85635182j]


In [130]:
#2.3
#define r_2_mag as the most reasonable root
#now we can calculate slant angle
r_2_mag = 9228.07608707

a_1 = 6 * ( D[2][0] * ( tau_1 / tau_3 ) + D[1][0] * ( tau / tau_3 ) ) * r_2_mag**3 + mu_E * D[2][0] * (tau**2 - tau_1**2) * ( tau_1 / tau_3 )
b_1 = 6 * r_2_mag**3 + mu_E * ( tau**2 - tau_3**2 )
rho_1 = (1/D_0) * ( a_1 / b_1 - D[0][0] )

a_3 = 6 * ( D[0][2] * ( tau_3 / tau_1 ) - D[1][2] * ( tau / tau_1 ) ) * r_2_mag**3 + mu_E * D[0][2] * (tau**2 - tau_1**2) * ( tau_3 / tau_1 )
b_3 = (6 * r_2_mag**3 + mu_E * ( tau**2 - tau_1**2 ) )
rho_3 = (1/D_0) * ( a_3 / b_3 - D[2][2] )

rho_2 = A + mu_E * B / r_2_mag**3

print(rho_1, rho_2, rho_3)

3622.9875928841643 3848.4430843045398 4171.666859661507


In [67]:
#calculate radius
r_1 = R_1 + rho_1 * rho_1_hat
r_2 = R_2 + rho_2 * rho_2_hat
r_3 = R_3 + rho_3 * rho_3_hat
print(r_1, r_2, r_3)

[6085.40619165 5896.46736948 3525.00534268] [5649.7702502  6520.84378001 3273.54254368] [5175.29707537 7119.79806009 2991.4505016 ]


In [68]:
#2.4
#calculate v_2 from f and g estimates
f1 = 1 - 0.5 * mu_E * tau_1**2 / r_2_mag**3
f3 = 1 - 0.5 * mu_E * tau_3**2 / r_2_mag**3
g1 = tau_1 - (1/6) * mu_E * tau_1**3 / r_2_mag**3
g3 = tau_3 - (1/6) * mu_E * tau_3**3 / r_2_mag**3
v_2 = (-f3 * r_1 + f1 * r_3) / (f1 * g3 - f3 * g1)
print(v_2)

[-3.83334281  5.15756953 -2.2473264 ]


In [125]:
#2.5-10
#calculate orbital elements
K = np.array([0,0,1])
I = np.array([1,0,0])
r = r_2
v = v_2
r_mag = la.norm(r)
v_mag = la.norm(v)
h = np.cross(r,v)
e = ( np.cross(v,h) / mu_E ) - r / r_mag
n = np.cross(K,h)
i = np.arccos( np.dot(h,K) / la.norm(h))
lil_omega = np.arccos( np.dot(n,e) / (la.norm(e) * la.norm(n)) )
big_omega = np.arccos(np.dot(n,I)/ la.norm(n))
f = np.arccos(np.dot(r_2,e) / (r_2_mag * la.norm(e)))
a = 1 / ( (2/r_mag) - ((v_mag**2)/mu_E) )

In [128]:
#check if any elements need to be adjusted
print(n, np.dot(r_2, v_2))
#n.j < 0, need to adjust big omega
big_omega = 2 * np.pi - big_omega

[  -148.26706323 -31537.98766479      0.        ] 4617.480404370952


In [129]:
#print final values
print('semimajor axis: ', a, '\n', 'eccentricity: ', la.norm(e), '\n', 'inclination: ', np.degrees(i), '\n', 'RAAN: ', np.degrees(big_omega))
print('argument of periapsis: ', np.degrees(lil_omega), '\n', 'true anomaly: ', np.degrees(f))

semimajor axis:  9954.293837187999 
 eccentricity:  0.10342160428778187 
 inclination:  30.22418490345657 
 RAAN:  269.73064183832565
argument of periapsis:  85.68839870993905 
 true anomaly:  49.50595785407631
